# Process SRM differential expression for Agora
This script combines SRM differential expression data ([paper](https://doi.org/10.1038%2Fs41591-020-0815-6)) and formats it to be consistent with LFQ and TMT proteomics differential expression data. 

SRM data was collected in two batches ("rounds"). Round 1 tested a single peptide per gene, while round 2 tested multiple peptides for multiple genes. The two rounds were analyzed separately using an ANOVA with post-hoc analysis using TukeyHSD, using the same processing steps as the LFQ and TMT data analysis for Agora. 

The processed ANOVA data is provided by the researchers on Synapse ([round1](https://www.synapse.org/#!Synapse:syn21448389), [round2](https://www.synapse.org/#!Synapse:syn21448395)), however they did not include confidence intervals or adjusted p-values, which Agora requires. 

To obtain confidence intervals, we instead ingest the raw data ([round1](https://www.synapse.org/#!Synapse:syn21448381), [round2](https://www.synapse.org/#!Synapse:syn21448378)) and partially replicate their post-hoc analysis (pairwise comparisons using TukeyHSD). The end result produces equivalent log2-fold-changes and p-values to the original processed data but also includes confidence intervals as calculated by TukeyHSD. The code the researchers used for their analysis, which we used for reference, is on Synapse [here](https://www.synapse.org/#!Synapse:syn21448382) and [here](https://www.synapse.org/#!Synapse:syn21448383). 

This script then adjusts the p-values for each protein using FDR correction, as was done for LFQ and TMT. This script also looks up the Ensembl gene IDs matching each peptide's UniProt ID, and then formats the data frame to match the format of LFQ and TMT data for Agora. For genes with multiple peptides tested, we use the peptide with the smallest corrected p-value in the final data frame, to get a single value per gene. 

## Install notes
This script requires install of several libraries:

```
pip install openpyxl statsmodels unipressed
```

and also assumes that Python is running in the environment created by [installing agora-data-tools](https://github.com/Sage-Bionetworks/agora-data-tools#locally).

In [1]:
import pandas as pd
import numpy as np
import statsmodels.stats.multicomp as mc
import statsmodels.stats.multitest as mt
from unipressed import IdMappingClient
import time
import re
import synapseclient
import agoradatatools.etl.extract as extract
import preprocessing_utils

# Step 1: Get and clean raw data and metadata
The data is downloaded from Synapse, and some gene names are fixed to be consistent across round 1 and round 2. Additionally, two outlier samples are dropped from round 2 data as was done in the [original analysis script](https://www.synapse.org/#!Synapse:syn21448383) for round 2. 

In [2]:
syn = synapseclient.Synapse()
syn.login(silent=True)

round1 = extract.get_entity_as_df(syn_id="syn21448381", source="csv", syn=syn)
round1 = round1.rename(columns={"ProjID": "GeneName"})

round2 = extract.get_entity_as_df(syn_id="syn21448378", source="csv", syn=syn)
round2 = round2.rename(columns={"ProjID": "GeneName"})

metadata = extract.get_entity_as_df(syn_id="syn21448380", source="csv", syn=syn)
metadata = metadata[["projid", "Group"]]
metadata = metadata.loc[
    (metadata["Group"] != "Exclude") & (metadata["Group"] != "SRMcontrol")
]

uniprot_mapping = extract.get_entity_as_df(syn_id="syn52850588", source="csv", syn=syn)


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.0.0) is available. Your version (3.1.1) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.0.0 release notes

https://python-docs.synapse.org/news/



Some peptide names are mis-matched between files. This mapping comes from the peptide info spreadsheet and from notes provided by the researchers. Some not listed here are still mis-matched after the re-map but we don't have any more information.

In [3]:
remaps = {
    "bA": "APP_3",
    "bA38": "APP_5",
    "bA42": "APP_6",
    "tau_AT8_s202": "MAPT_2",
    "HLA_B_2": "HLA-B_2",
    "HLA_B_5": "HLA-B_5",
}

round1["GeneName"] = round1["GeneName"].replace(remaps)
round2["GeneName"] = round2["GeneName"].replace(remaps)

The dataframes are arranged so rows are genes and columns are samples. For the data manipulation below, it is easier for the data to be transposed. Two outlier samples are dropped from round 2 data.

In [4]:
round1 = round1.set_index("GeneName").transpose()
round2 = round2.set_index("GeneName").transpose()
round2 = round2.drop(index=["X10300914", "X15196262"])

# Step 2: Get confidence intervals and adjusted p-values

Create a function to perform pairwise TukeyHSD for each gene separately. This code is equivalent to and produces identical results to the post-hoc analysis done to produce the [round 1 ANOVA results](https://www.synapse.org/#!Synapse:syn21448389) and [round2 ANOVA results](https://www.synapse.org/#!Synapse:syn21448395), which were originally processed using R scripts for [round 1](https://www.synapse.org/#!Synapse:syn21448382) and [round 2](https://www.synapse.org/#!Synapse:syn21448383). The only difference is that we also include the confidence intervals calculated by `pairwise_tukeyhsd`, which are needed for Agora. 

We also have to explicitly create a column for the gene name for use in merging later on. In case of testing multiple peptides for a gene, the index/gene will contain an underscore plus a numeric value (e.g `APOD_2`), so we strip off everything after the underscore to get the gene name. 

In [5]:
regex = re.compile(r"_.*")


def do_tukeyhsd_per_gene(gene, input_df, metadata):
    # Merge the metadata and the gene expression column, removing samples with NA data
    df = pd.merge(
        left=metadata,
        right=input_df[gene],
        how="inner",
        left_on="projid",
        right_index=True,
        validate="one_to_one",
    )
    df = df.dropna()

    # The output of pairwise_tukeyhsd doesn't provide an easy way to extract specific comparisons,
    # so this puts the output into a pandas DataFrame and pulls out which index corresponds to
    # 'AD vs Control'.
    result = mc.pairwise_tukeyhsd(endog=df[gene], groups=df["Group"], alpha=0.05)
    res_df = pd.DataFrame(result.summary().data, columns=result.summary().data[0])
    res_df = res_df.drop(index=0)
    ind = np.where((res_df["group1"] == "AD") & (res_df["group2"] == "Control"))[0][0]

    # The diff and confidence interval values need to be opposite sign from what is
    # in the result because the result is reporting Control-AD, not AD-Control
    results_dict = {
        "GeneName": regex.sub("", gene),
        "Log2_FC": -result.meandiffs[ind],
        "CI_Upr": -result.confint[ind][0],
        "CI_Lwr": -result.confint[ind][1],
        "PVal": result.pvalues[ind],
    }

    return pd.DataFrame(results_dict, index=[gene])

Run TukeyHSD on each round separately. We do this so we can adjust the p-values for each round separately, since they were treated as two different datasets originally.

In [6]:
round1_list = [do_tukeyhsd_per_gene(gene, round1, metadata) for gene in round1.columns]
round1_res = pd.concat(round1_list, axis=0)

round2_list = [do_tukeyhsd_per_gene(gene, round2, metadata) for gene in round2.columns]
round2_res = pd.concat(round2_list, axis=0)

Adjust p-values for multiple testing *before* combining the data frames, as they were run as separate ANOVAs. Uses FDR correction, which is what was used for LFQ and TMT data processing.

In [7]:
def adjust_pvals(df):
    (_, adj_p, _, _) = mt.multipletests(pvals=df["PVal"], alpha=0.05, method="fdr_bh")
    df["Cor_PVal"] = adj_p
    return df


round1_res = adjust_pvals(round1_res)
round2_res = adjust_pvals(round2_res)

Combine round 1 and round 2 into one data frame. Only 4 genes overlap between them, but unlike round 1, which selected a single peptide variant per gene, they ran differential expression on individual peptides and included all peptide variants in the round 2 table. For Agora, we will pick the variant/gene with the smallest corrected p-value between the two rounds. 

In [8]:
diffexp = pd.concat([round1_res, round2_res], axis=0, ignore_index=True)
rows = diffexp.groupby("GeneName").agg({"Cor_PVal": "idxmin"})
diffexp = diffexp.loc[rows["Cor_PVal"]]

diffexp

,GeneName,Log2_FC,CI_Upr,CI_Lwr,PVal,Cor_PVal
0,AK4,0.081583,0.124460,0.038705,2.639619e-05,1.607768e-04
67,AMPD2,0.145948,0.198572,0.093324,3.555050e-10,6.043584e-09
1,ANKRD40,-0.021340,0.077472,-0.120152,8.679973e-01,9.996966e-01
2,AP2A2,-0.035770,-0.003280,-0.068261,2.674922e-02,6.179993e-02
68,AP2B1,-0.003357,0.039702,-0.046415,9.817129e-01,9.998368e-01
...,...,...,...,...,...,...
184,VDAC2,-0.002774,0.056417,-0.061966,9.933509e-01,9.998368e-01
64,VGF,-0.469397,-0.355691,-0.583104,0.000000e+00,0.000000e+00
65,VPS35,0.004000,0.060699,-0.052700,9.850006e-01,9.996966e-01
66,VSNL1,-0.013745,0.014687,-0.042178,4.928746e-01,7.178826e-01


# Step 3: Get peptide IDs and Ensembl IDs for each gene
This step queries UniProt for the Ensembl IDs associated with the UniProt IDs in this dataset. 

In [9]:
peptide_info = uniprot_mapping.dropna(subset=["UniProtAC"])
uniprot_ids = peptide_info["UniProtAC"].drop_duplicates()
request = IdMappingClient.submit(
    source="UniProtKB_AC-ID", dest="Ensembl", ids=uniprot_ids
)

found = False
while not found:
    time.sleep(2)
    try:
        ensembl_ids = list(request.each_result())
        found = True
    except:
        print("Waiting for response from UniProt...")

Rename columns to match, and get rid of the Ensembl version at the end of each ID

In [10]:
ensembl_ids = pd.DataFrame(ensembl_ids).rename(
    columns={"from": "UniProtAC", "to": "ENSG"}
)
ensembl_ids["ENSG"] = ensembl_ids["ENSG"].str.extract(r"(\w+)")

Filter out HASGs by querying Biomart for chromosome information and removing genes with HASG chromosomes notation. 

In [11]:
attributes = ["ensembl_gene_id", "chromosome_name"]
filters = {"ensembl_gene_id": set(ensembl_ids["ENSG"])}

biomart_result = preprocessing_utils.manual_query_biomart(
    attributes=attributes, filters=filters
)
biomart_result = biomart_result.rename(
    columns={"Gene stable ID": "ENSG", "Chromosome/scaffold name": "chromosome_name"}
)
biomart_result

,ENSG,chromosome_name
0,ENSG00000005893,X
1,ENSG00000006125,17
2,ENSG00000007376,16
3,ENSG00000008056,X
4,ENSG00000010310,19
...,...,...
246,ENSG00000285390,HSCHR1_8_CTG3
247,ENSG00000288283,HG109_PATCH
248,ENSG00000288299,HG2111_PATCH
249,ENSG00000288301,HG2087_PATCH


In [12]:
valid_ids = preprocessing_utils.filter_hasgs(
    df=biomart_result, chromosome_name_column="chromosome_name"
)
valid_ids

,ENSG,chromosome_name
0,ENSG00000005893,X
1,ENSG00000006125,17
2,ENSG00000007376,16
3,ENSG00000008056,X
4,ENSG00000010310,19
...,...,...
224,ENSG00000224051,1
225,ENSG00000234745,6
226,ENSG00000241468,7
227,ENSG00000265681,18


Update the list of Ensembl IDs to only contain valid (non-HASG) genes.

In [13]:
ensembl_ids = ensembl_ids.loc[ensembl_ids["ENSG"].isin(valid_ids["ENSG"])]
ensembl_ids

,UniProtAC,ENSG
0,Q9UKV3,ENSG00000100813
1,Q6AI12,ENSG00000154945
2,O94973,ENSG00000183020
3,P02649,ENSG00000130203
4,P05067,ENSG00000142192
...,...,...
249,P45880,ENSG00000165637
250,O95619,ENSG00000127337
251,P49750,ENSG00000119596
252,Q9H0M4,ENSG00000078487


Add the IDs to the peptide info data frame

In [14]:
peptide_info = peptide_info.merge(ensembl_ids, how="left", on="UniProtAC")

Remove unneeded columns from peptide_info and rename remaining columns to match intended output

In [15]:
peptide_info = peptide_info[["UniProtAC", "Gene", "ENSG"]].drop_duplicates()
peptide_info = peptide_info.rename(
    columns={"UniProtAC": "UniProtID", "Gene": "GeneName"}
)
peptide_info

,UniProtID,GeneName,ENSG
0,Q9UKV3,ACIN1,ENSG00000100813
2,Q6AI12,ANKRD40,ENSG00000154945
4,O94973,AP2A2,ENSG00000183020
6,P02649,APOE,ENSG00000130203
8,P05067,APP,ENSG00000142192
...,...,...,...
452,P45880,VDAC2,ENSG00000165637
454,O95619,YEATS4,ENSG00000127337
456,P49750,YLPM1,ENSG00000119596
458,Q9H0M4,ZCWPW1,ENSG00000078487


# Step 4: Restructure round 1 & 2 data to match other proteomics data
Data must have the same columns and format as [the LFQ differential expression data](https://www.synapse.org/#!Synapse:syn18689335).

First, merge in the peptide info to get Ensembl IDs and UniProt IDs. This data also needs a tissue column to match LFQ and TMT.  

In [16]:
diffexp_final = diffexp.merge(peptide_info, how="inner", on="GeneName")
diffexp_final["Tissue"] = "DLPFC"

In [17]:
diffexp_final

,GeneName,Log2_FC,CI_Upr,CI_Lwr,PVal,Cor_PVal,UniProtID,ENSG,Tissue
0,AK4,0.081583,0.124460,0.038705,2.639619e-05,1.607768e-04,P27144,ENSG00000162433,DLPFC
1,AMPD2,0.145948,0.198572,0.093324,3.555050e-10,6.043584e-09,Q01433,ENSG00000116337,DLPFC
2,ANKRD40,-0.021340,0.077472,-0.120152,8.679973e-01,9.996966e-01,Q6AI12,ENSG00000154945,DLPFC
3,AP2A2,-0.035770,-0.003280,-0.068261,2.674922e-02,6.179993e-02,O94973,ENSG00000183020,DLPFC
4,AP2B1,-0.003357,0.039702,-0.046415,9.817129e-01,9.998368e-01,P63010,ENSG00000006125,DLPFC
...,...,...,...,...,...,...,...,...,...
121,VAT1,0.095231,0.185810,0.004653,3.662795e-02,8.180242e-02,Q99536,ENSG00000108828,DLPFC
122,VDAC2,-0.002774,0.056417,-0.061966,9.933509e-01,9.998368e-01,P45880,ENSG00000165637,DLPFC
123,VGF,-0.469397,-0.355691,-0.583104,0.000000e+00,0.000000e+00,O15240,ENSG00000128564,DLPFC
124,VPS35,0.004000,0.060699,-0.052700,9.850006e-01,9.996966e-01,Q96QK1,ENSG00000069329,DLPFC


As in the other proteomics data, the unique ID is "\<GeneName\>|\<UniProtID\>"

In [18]:
diffexp_final["UniqID"] = diffexp_final["GeneName"] + "|" + diffexp_final["UniProtID"]

Put the necessary columns in the correct order

In [19]:
diffexp_final = diffexp_final[
    [
        "UniqID",
        "GeneName",
        "UniProtID",
        "ENSG",
        "Tissue",
        "Log2_FC",
        "CI_Upr",
        "CI_Lwr",
        "PVal",
        "Cor_PVal",
    ]
]

In [20]:
diffexp_final

,UniqID,GeneName,UniProtID,ENSG,Tissue,Log2_FC,CI_Upr,CI_Lwr,PVal,Cor_PVal
0,AK4|P27144,AK4,P27144,ENSG00000162433,DLPFC,0.081583,0.124460,0.038705,2.639619e-05,1.607768e-04
1,AMPD2|Q01433,AMPD2,Q01433,ENSG00000116337,DLPFC,0.145948,0.198572,0.093324,3.555050e-10,6.043584e-09
2,ANKRD40|Q6AI12,ANKRD40,Q6AI12,ENSG00000154945,DLPFC,-0.021340,0.077472,-0.120152,8.679973e-01,9.996966e-01
3,AP2A2|O94973,AP2A2,O94973,ENSG00000183020,DLPFC,-0.035770,-0.003280,-0.068261,2.674922e-02,6.179993e-02
4,AP2B1|P63010,AP2B1,P63010,ENSG00000006125,DLPFC,-0.003357,0.039702,-0.046415,9.817129e-01,9.998368e-01
...,...,...,...,...,...,...,...,...,...,...
121,VAT1|Q99536,VAT1,Q99536,ENSG00000108828,DLPFC,0.095231,0.185810,0.004653,3.662795e-02,8.180242e-02
122,VDAC2|P45880,VDAC2,P45880,ENSG00000165637,DLPFC,-0.002774,0.056417,-0.061966,9.933509e-01,9.998368e-01
123,VGF|O15240,VGF,O15240,ENSG00000128564,DLPFC,-0.469397,-0.355691,-0.583104,0.000000e+00,0.000000e+00
124,VPS35|Q96QK1,VPS35,Q96QK1,ENSG00000069329,DLPFC,0.004000,0.060699,-0.052700,9.850006e-01,9.996966e-01


Write to csv and upload to Synapse.

In [23]:
diffexp_final.to_csv("../../output/SRM_diff_expr.csv", index=False)
file = synapseclient.File("../../output/SRM_diff_expr.csv", parent="syn19275585")  # syn19275585 is the Agora Proteomics Analysis folder
file = syn.store(
    file,
    used=[
        "syn21448381",
        "syn21448378",
        "syn21448380",
        "syn52850588",
        "https://doi.org/10.1038%2Fs41591-020-0815-6",
    ],
    forceVersion=False,
)


##################################################
 Uploading file to your external S3 storage 
##################################################

INFO: 2024-01-23 15:47:21 | synapseclient_default | 
##################################################
 Uploading file to your external S3 storage 
##################################################

